In [1]:
import sys
sys.path.append('..')
sys.path.append('../src')
sys.path.append('../data')

%load_ext autoreload

In [2]:
import json
from tqdm import tqdm

# Generate a standard dataset to evaluate

In [7]:
import random
random.seed(42)
data = json.load(open("../data/counterfact.json"))
final_data = []
# shuffle data
random.shuffle(data)
for i, d in tqdm(enumerate(data), total=len(data)):
    target_new = " " + d["requested_rewrite"]["target_new"]["str"]
    target_true = " " + d["requested_rewrite"]["target_true"]["str"]
        
    for p in d["attribute_prompts"]:
        template = "{}: " + p + "{}" + ". " + p
        final_data.append(
            {
            "base_prompt": p,
            "template": template,
            "true": target_new,
            "false": target_true,
            }
        )
        final_data[-1]["prompt"] = template.format("Redefine", final_data[-1]["false"])
    for p in d["neighborhood_prompts"]:
        template = "{}: " + p + "{}" + ". " + p
        final_data.append(
            {
            "base_prompt": p,
            "template": template,
            "true": target_true,
            "false": target_new,
            }
        )
        final_data[-1]["prompt"] = template.format("Redefine", final_data[-1]["false"])
        
unique_strs = set(json.dumps(d) for d in final_data)
final_data = [json.loads(d) for d in unique_strs]
print(len(final_data))
        

100%|██████████| 21919/21919 [00:00<00:00, 57692.49it/s]


284753


In [8]:
json.dump(final_data, open("../data/full_data.json", "w"), indent=2)

## Load HF model

In [18]:
from src.score_models import HFDataset, EvaluateMechanism
from transformers import AutoTokenizer, AutoModelForCausalLM
%load_ext autoreload
%autoreload 2

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125M")
model = AutoModelForCausalLM.from_pretrained("facebook/opt-125M")
model = model.cuda()
dataset =HFDataset("../data/full_data_sampled_gpt2.json", tokenizer=tokenizer, slice=100)

# evaluator = EvaluateMechanism("gpt2", dataset)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload





Tokenizing prompts: 100%|██████████| 100/100 [00:00<00:00, 8937.36it/s]


In [22]:
dataset.set_len(21, orthogonal=True, model=model)










100%|██████████| 21/21 [00:00<00:00, 80.32it/s]


In [28]:
dataset[1]

{'prompt': 'Redefine: Toyota Sprinter Carib is produced by Cand. Toyota Sprinter Carib is produced by',
 'input_ids': tensor([    2, 15638,  4550,   833,    35,  7261, 14933,  8007, 34093,    16,
          2622,    30, 11323,     4,  7261, 14933,  8007, 34093,    16,  2622,
            30]),
 'target': tensor([ 7261, 11323]),
 'obj_pos': 12}

In [30]:
tokenizer.decode(dataset[1]["input_ids"])

'</s>Redefine: Toyota Sprinter Carib is produced by Cand. Toyota Sprinter Carib is produced by'

In [33]:
tokenizer.decode(11323)

' Cand'

In [61]:
one = model.get_input_embeddings()(tokenizer.encode(" kidnapped", return_tensors="pt"))

torch.Size([1, 1, 768])

In [64]:
tokenizer.encode("50")

[1120]

In [9]:
two = model.get_input_embeddings()(tokenizer.encode(" kidnapped", return_tensors="pt"))

In [25]:
token = tokenizer.encode("Apple", return_tensors="pt")
print(token.shape)
emb = model.get_input_embeddings()(torch.tensor(token))

torch.Size([1, 1])


/tmp/ipykernel_86529/710277193.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = model.get_input_embeddings()(torch.tensor(token))


In [26]:
import einops
emb = einops.rearrange(emb, "b s e -> (b s e)")

In [28]:
emb.shape

torch.Size([768])

In [ ]:
# find orthogonal vecotor to emb using torch


In [9]:
tokenizer.decode(model.generate(tokenizer.encode("Redefine: ", return_tensors="pt"), max_length=100)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Redefine: \xa0The first thing I did was to make a list of all the things I would like to see in the game. I wanted to make sure that the game was as good as possible, and that it was as good as possible for the players. I wanted to make sure that the game was as good as possible for the players. I wanted to make sure that the game was as good as possible for the players. I wanted to make sure that the game was as good'

In [37]:
import torch
dataset.set_len(134)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=False)

[16849] [3340]
tensor([16849]) tensor([3340])
tensor([[16849,  3340]])


In [11]:
import torch

In [3]:
from src.model import WrapHookedTransformer
# from transformers import AutoTokenizer, AutoModelForCausalLM

In [4]:
modeltl = WrapHookedTransformer.from_pretrained("opt-125m")

Loaded pretrained model opt-125m into HookedTransformer


In [5]:
modeltl.to_string(15675)

' Apollo'

In [15]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")
model = AutoModelForCausalLM.from_pretrained("facebook/opt-125m")

In [6]:
modeltl.predict("Redefine: Alvin Lustig originates from Paris. Alvin Lustig originates from")

(tensor([16.1959], device='cuda:0'), [' Paris'])

In [16]:
model(torch.tensor(tokenizer.encode("Redefine: Lucky Dime Caper was created by Apple. Lucky Dime Caper was created by")).unsqueeze(0))["logits"][0, -1, :].argmax()

tensor(1257)

In [20]:
tokenizer.decode(1257)

' Apple'

In [19]:
modeltl.to_tokens(" Apple")

tensor([[   2, 1257]], device='cuda:0')

In [7]:
for batch in dataloader:
    print(batch["input_ids"].shape)

In [1]:
import sys
sys.path.append('..')
sys.path.append('../src')
sys.path.append('../data')
import json
data = json.load(open("../data/full_data.json"))

In [24]:
data[100]

{'base_prompt': 'MLS player ID, located in',
 'template': '{}: MLS player ID, located in{}. MLS player ID, located in',
 'target_true': ' Canada',
 'target_new': ' India',
 'prompt': 'Redefine: MLS player ID, located in India. MLS player ID, located in'}

In [11]:
from src.model import WrapHookedTransformer
model = WrapHookedTransformer.from_pretrained("opt-125m")

Loaded pretrained model opt-125m into HookedTransformer
